In [1]:
!pip install openai-agents
!pip install openai
!pip install pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.9/210.9 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.4/144.4 kB 8.4 MB/s eta 0:00:00


In [11]:
from pydantic import BaseModel
from agents import Agent, ModelSettings, TResponseInputItem, Runner, RunConfig
from openai.types.shared.reasoning import Reasoning

class TriageRequestSchema(BaseModel):
  classification: str


class ApprovalAgentSchema(BaseModel):
  emailFrom: str
  defaultTo: str
  defaultSubject: str
  defaultBody: str


triage_request = Agent(
  name="Triage request",
  instructions="""Classify the user's request based on whether two documents have been provided recently in the conversation, and whether the user is asking a particular question.

If two documents are provided and there's no user question , respond with \"compare\".
If two documents are provided and there is a user question , respond with \"answer_question\".
If only one doc has been provided, or no docs have been provided, respond with \"request_upload\"""",
  model="gpt-4.1",
  output_type=TriageRequestSchema,
  model_settings=ModelSettings(
    temperature=1,
    top_p=1,
    max_tokens=2048,
    store=True
  )
)


propose_reconciliation = Agent(
  name="Propose reconciliation",
  instructions="Given the differences between the two documents, assemble a single option for how to reconcile the difference. If no order has been described, consider the first document the user's version and the second document the potential set of changes returned back to the user. The proposal you create will be sent to the user for approval.",
  model="gpt-5",
  model_settings=ModelSettings(
    store=True,
    reasoning=Reasoning(
      effort="minimal",
      summary="auto"
    )
  )
)


approval_agent = Agent(
  name="Approval agent",
  instructions="""Explain your approval reasoning. Help the user draft a proper response by filling out this data schema:

{
  emailFrom: 'user@test.com',
  defaultTo: 'user@test.com',
  defaultSubject: 'Document comparison proposal',
  defaultBody: \"Hey there, \n\nHope you're doing well! Just wanted to check in and see if there are any updates on the ChatKit roadmap. We're excited to see what's coming next and how we can make the most of the upcoming features.\n\nEspecially curious to see how you support widgets!\n\nBest,\",
}""",
  model="gpt-5-mini",
  output_type=ApprovalAgentSchema,
  model_settings=ModelSettings(
    store=True,
    reasoning=Reasoning(
      effort="low",
      summary="auto"
    )
  )
)


rejection_agent = Agent(
  name="Rejection agent",
  instructions="Explain your rejection reasoning.",
  model="gpt-5",
  model_settings=ModelSettings(
    store=True,
    reasoning=Reasoning(
      effort="low",
      summary="auto"
    )
  )
)


retry_agent = Agent(
  name="Retry agent",
  instructions="The user has not uploaded the required two documents for comparison. Suggest that they upload a total of two documents, using the paperclip icon.",
  model="gpt-5-nano",
  model_settings=ModelSettings(
    store=True,
    reasoning=Reasoning(
      effort="minimal",
      summary="auto"
    )
  )
)


provide_explanation = Agent(
  name="Provide explanation",
  instructions="Use the information in the uploaded documents to answer the user's question.",
  model="gpt-5-nano",
  model_settings=ModelSettings(
    store=True,
    reasoning=Reasoning(
      effort="minimal",
      summary="auto"
    )
  )
)


def approval_request(message: str):
  # TODO: Implement
  return True

class WorkflowInput(BaseModel):
  input_as_text: str


# Main code entrypoint
async def run_workflow(workflow_input: WorkflowInput):
  state = {

  }
  workflow = workflow_input.model_dump()
  conversation_history: list[TResponseInputItem] = [
    {
      "role": "user",
      "content": [
        {
          "type": "input_text",
          "text": workflow["input_as_text"]
        }
      ]
    }
  ]
  triage_request_result_temp = await Runner.run(
    triage_request,
    input=[
      *conversation_history
    ],
    run_config=RunConfig(trace_metadata={
      "__trace_source__": "agent-builder",
      "workflow_id": "wf_68e7d3ecffdc81909d6bd4ef54e13f97041f23f4d1d6d373"
    })
  )

  conversation_history.extend([item.to_input_item() for item in triage_request_result_temp.new_items])

  triage_request_result = {
    "output_text": triage_request_result_temp.final_output.json(),
    "output_parsed": triage_request_result_temp.final_output.model_dump()
  }
  if triage_request_result["output_parsed"]["classification"] == "compare":
    propose_reconciliation_result_temp = await Runner.run(
      propose_reconciliation,
      input=[
        *conversation_history
      ],
      run_config=RunConfig(trace_metadata={
        "__trace_source__": "agent-builder",
        "workflow_id": "wf_68e7d3ecffdc81909d6bd4ef54e13f97041f23f4d1d6d373"
      })
    )

    conversation_history.extend([item.to_input_item() for item in propose_reconciliation_result_temp.new_items])

    propose_reconciliation_result = {
      "output_text": propose_reconciliation_result_temp.final_output_as(str)
    }
    approval_message = f"Please review the proposal {propose_reconciliation_result["output_text"]}"

    if approval_request(approval_message):
        approval_agent_result_temp = await Runner.run(
          approval_agent,
          input=[
            *conversation_history
          ],
          run_config=RunConfig(trace_metadata={
            "__trace_source__": "agent-builder",
            "workflow_id": "wf_68e7d3ecffdc81909d6bd4ef54e13f97041f23f4d1d6d373"
          })
        )

        conversation_history.extend([item.to_input_item() for item in approval_agent_result_temp.new_items])

        approval_agent_result = {
          "output_text": approval_agent_result_temp.final_output.json(),
          "output_parsed": approval_agent_result_temp.final_output.model_dump()
        }
    else:
        rejection_agent_result_temp = await Runner.run(
          rejection_agent,
          input=[
            *conversation_history
          ],
          run_config=RunConfig(trace_metadata={
            "__trace_source__": "agent-builder",
            "workflow_id": "wf_68e7d3ecffdc81909d6bd4ef54e13f97041f23f4d1d6d373"
          })
        )

        conversation_history.extend([item.to_input_item() for item in rejection_agent_result_temp.new_items])

        rejection_agent_result = {
          "output_text": rejection_agent_result_temp.final_output_as(str)
        }
  elif triage_request_result["output_parsed"]["classification"] == "answer_question":
    provide_explanation_result_temp = await Runner.run(
      provide_explanation,
      input=[
        *conversation_history
      ],
      run_config=RunConfig(trace_metadata={
        "__trace_source__": "agent-builder",
        "workflow_id": "wf_68e7d3ecffdc81909d6bd4ef54e13f97041f23f4d1d6d373"
      })
    )

    conversation_history.extend([item.to_input_item() for item in provide_explanation_result_temp.new_items])

    provide_explanation_result = {
      "output_text": provide_explanation_result_temp.final_output_as(str)
    }
  else:
    retry_agent_result_temp = await Runner.run(
      retry_agent,
      input=[
        *conversation_history
      ],
      run_config=RunConfig(trace_metadata={
        "__trace_source__": "agent-builder",
        "workflow_id": "wf_68e7d3ecffdc81909d6bd4ef54e13f97041f23f4d1d6d373"
      })
    )

    conversation_history.extend([item.to_input_item() for item in retry_agent_result_temp.new_items])

    retry_agent_result = {
      "output_text": retry_agent_result_temp.final_output_as(str)
    }


In [15]:
import os, json, asyncio, threading
from pathlib import Path
from typing import List, Dict, Any, Tuple
from google.colab import userdata

# -------- INPUTS (edit these) --------
USER_TEXT: str = ""           # or "" if you only want to send files
FILE_PATHS: List[str] = ["/content/lattes.pdf", "/content/linkedin.pdf"]          # e.g. ["/sample.pdf", "/image.png"]
# -------------------------------------

# 0) Check API key
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
if not os.getenv("OPENAI_API_KEY"):
    raise RuntimeError("OPENAI_API_KEY is not set in the environment.")

# 1) Imports (Agents SDK)
from openai import OpenAI
from agents import Agent, Runner, RunConfig, TResponseInputItem

# 2) Discover Agents defined in Cell 2 (generic)
def _discover_agents() -> List[Tuple[str, Agent]]:
    import __main__
    agents = []
    for name, val in vars(__main__).items():
        if isinstance(val, Agent):
            agents.append((name, val))
    if not agents:
        raise RuntimeError("No Agent instances found. Make sure Cell 2 created at least one Agent.")
    # Heuristic: place summarizers last
    def _key(pair):
        name, _ = pair
        is_summarizer = int(any(k in name.lower() for k in ("summary","summarize","display","final")))
        return (is_summarizer, name.lower())
    agents.sort(key=_key)
    return agents

# 3) Build Responses-style inputs (text + files)
def _build_items(user_text: str, file_paths: List[str]) -> List[TResponseInputItem]:
    content: List[Dict[str, Any]] = []
    user_text = (user_text or "").strip()
    if user_text:
        content.append({"type": "input_text", "text": user_text})
    # upload files -> input_file items
    if file_paths:
        client = OpenAI()
        for p in file_paths:
            path = Path(p).expanduser().resolve()
            if not path.exists() or not path.is_file():
                raise FileNotFoundError(f"File not found: {path}")
            with path.open("rb") as f:
                up = client.files.create(file=(path.name, f.read()), purpose="assistants")
            content.append({"type": "input_file", "file_id": up.id})
    if not content:
        raise ValueError("Provide USER_TEXT and/or FILE_PATHS.")
    # one user message with all content
    return [{"role": "user", "content": content}]

# 4) Run one agent with approval loop
async def _run_with_approvals(agent: Agent, input_payload):
    print(f"\n→ Running agent: {agent.name!r}")
    result = await Runner.run(agent, input=input_payload, run_config=RunConfig())
    # approvals loop
    while True:
        interruptions = getattr(result, "interruptions", []) or []
        approvals = [intr for intr in interruptions
                     if getattr(intr, "type", None) in ("tool_approval_item", "mcp_approval_item", "approval_item")]
        if not approvals:
            break
        print("\n=== Approvals requested ===")
        for idx, intr in enumerate(approvals, 1):
            raw = getattr(intr, "raw_item", None)
            tool_name = getattr(raw, "name", "tool")
            tool_args = getattr(raw, "arguments", {})
            print(f"[{idx}] {tool_name} args={tool_args}")
            resp = input("Approve? [y/N]: ").strip().lower()
            if resp in ("y","yes"):
                result.state.approve(intr)
                print(" → approved.")
            else:
                result.state.reject(intr)
                print(" → rejected.")
        # resume with updated state
        result = await Runner.run(agent, input=result.state, run_config=RunConfig())
    # pretty output
    final_obj = getattr(result, "final_output", None)
    final_json = final_obj.json() if hasattr(final_obj, "json") else None
    final_parsed = final_obj.model_dump() if hasattr(final_obj, "model_dump") else None
    print("\n=== Agent final output ===")
    print(json.dumps({
        "final_output_text": final_json,
        "final_output_parsed": final_parsed,
        "trace_id": getattr(result, "trace_id", None),
        "interruptions": [getattr(i, "type", None) for i in getattr(result, "interruptions", [])] if hasattr(result, "interruptions") else [],
    }, ensure_ascii=False, indent=2))
    return result

# 5) End-to-end: run all discovered agents in sequence, passing the evolved state forward
async def _main():
    agents_list = _discover_agents()
    print("Detected agents (execution order):", [name for name,_ in agents_list])
    items = _build_items(USER_TEXT, FILE_PATHS)

    # first agent gets the user message; subsequent agents receive the evolved state
    state_or_items = items
    for name, agent in agents_list:
        result = await _run_with_approvals(agent, state_or_items)
        state_or_items = result.state  # pass the whole state forward

    print("\n✅ Workflow complete.")

# 6) Run in a dedicated thread with its own asyncio loop (avoids notebook loop conflicts)
def _runner():
    asyncio.run(_main())

t = threading.Thread(target=_runner, daemon=False)
t.start()
t.join()


Detected agents (execution order): ['approval_agent', 'propose_reconciliation', 'provide_explanation', 'rejection_agent', 'retry_agent', 'triage_request', 'web_research_agent', 'summarize_and_display']

→ Running agent: 'Approval agent'


/tmp/ipython-input-2870606196.py:86: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  final_json = final_obj.json() if hasattr(final_obj, "json") else None
Exception in thread Thread-10 (_runner):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipython-input-2870606196.py", line 113, in _runner
  File "/usr/lib/python3.12/asyncio/runners.py", line 195, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/asyncio/base_events.py",


=== Agent final output ===
{
  "final_output_text": "{\"emailFrom\":\"user@test.com\",\"defaultTo\":\"user@test.com\",\"defaultSubject\":\"Document comparison proposal\",\"defaultBody\":\"Hi Éfrem,\\n\\nI reviewed the CV/summary you shared and I approve it for use/publication. Reasoning: the document is comprehensive and up to date (last updated 01/09/2025 per Lattes); it clearly lists education (PhD, MSc, BSc), current roles (Principal Product Manager at Pipefy; professor roles at FGV and Unieuro), relevant work history (Pipefy, Zeeplo, OLX, Globo, Loft, etc.), teaching and extension activities, research lines, publications, and contact information. Key strengths: clear product + AI focus, strong teaching experience, practical no-code/low-code expertise, and relevant publications and projects. \\n\\nSuggested minor edits before final distribution:\\n- Fix small typos/encoding issues (e.g., “Reponsável” -> \\\"Responsável\\\", remove duplicated punctuation or odd characters in heading